In [54]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode
from syft.core.frameworks.torch.mpc_tensor import _MPCTensor
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

me.add_workers([bob, alice, james])
bob.add_workers([me, alice, james])
alice.add_workers([me, bob, james])
james.add_workers([me, bob, alice])

def share(x):
    x_enc = spdz.encode(x)
    x_alice, x_bob = spdz.share(x_enc)
    x_alice.send(alice)
    x_bob.send(bob)
    x_pointer_tensor_dict = {alice: x_alice.child, bob: x_bob.child}
    x_gp = _GeneralizedPointerTensor(x_pointer_tensor_dict).on(x)
    x_mpc = _MPCTensor(x_gp)
    return x_mpc

x = share(torch.LongTensor([[1,2,3,4,5],[1,2,3,4,5]]))
y = share(torch.LongTensor([[2,2,2,2,2],[1,1,1,1,1]]))

z = x * y

z.get()


  2   4   6   8  10
  1   2   3   4   5
[syft.core.frameworks.torch.tensor.LongTensor of size 2x5]


  2   4   6   7  10
  0   2   3   4   5
[syft.core.frameworks.torch.tensor.LongTensor of size 2x5]

In [11]:
def generate_mpc_number_pair(n1, n2):
    x = torch.LongTensor([n1])
    y = torch.LongTensor([n2])
    x_enc = spdz.encode(x)
    y_enc = spdz.encode(y)
    x_alice, x_bob = spdz.share(x_enc)
    y_alice, y_bob = spdz.share(y_enc)
    x_alice.send(alice)
    x_bob.send(bob)
    y_alice.send(alice)
    y_bob.send(bob)
    x_pointer_tensor_dict = {alice: x_alice.child, bob: x_bob.child}
    y_pointer_tensor_dict = {alice: y_alice.child, bob: y_bob.child}
    x_gp = _GeneralizedPointerTensor(x_pointer_tensor_dict).on(x)
    y_gp = _GeneralizedPointerTensor(y_pointer_tensor_dict).on(y)
    x_mpc = _MPCTensor(x_gp)
    y_mpc = _MPCTensor(y_gp)
    return x_mpc, y_mpc

def mpc_sum(n1, n2):
    x_mpc, y_mpc = generate_mpc_number_pair(n1, n2)
    sum_mpc = x_mpc + y_mpc
    sum_mpc = sum_mpc.get()
    assert torch.eq(sum_mpc, torch.LongTensor([n1 + n2])).all()

def test_mpc_sum():
    self.mpc_sum(3, 5)
    self.mpc_sum(4, 0)
    self.mpc_sum(5, -5)
    self.mpc_sum(3, -5)
    self.mpc_sum(2 ** 24, 2 ** 12)

def mpc_mul(n1, n2):
    x_mpc, y_mpc = generate_mpc_number_pair(n1, n2)
    mul_mpc = x_mpc * y_mpc
    mul_mpc = mul_mpc.get()
    return mul_mpc
    assert torch.eq(mul_mpc, torch.LongTensor([n1 * n2])).all(), (mul_mpc, 'should be', torch.LongTensor([n1 * n2]))

def test_mpc_mul():
    mpc_mul(3, 5)
    mpc_mul(4, 0)
    mpc_mul(5, -5)
    mpc_mul(3, 5)
    mpc_mul(2 ** 12, 2 ** 12)

In [26]:
mpc_mul(3, 5)


 14
[syft.core.frameworks.torch.tensor.LongTensor of size 1]